julie.tsitron@parks.nyc.gov 2/20/2020

# Imports and Connections to DBs

In [1]:
pwd

'c:\\Projects\\Structures\\develop'

In [2]:
import pyproj

In [3]:
import geopandas

In [4]:
geopandas.__version__

'0.8.1'

In [5]:
pyproj.__version__

'2.6.1.post1'

In [6]:
# pyproj.

In [7]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

sys.path.append('../')
from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *

In [8]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [9]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [10]:
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                           ';Database=;Trusted_Connection=Yes;')

## Deal with Dates

### m/d/Y H:M:S format:

In [11]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [12]:
# structures_dev['COMMISSIONDATE']

In [13]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [14]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [15]:
# structures_dev.columns.values

# Delta Table from structuresdb

In [16]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [17]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [18]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
0,1,5679,R013-BLG3581,5153581,5055700001,444585,9.0,16.00,1931.0,NaN,None,U,current,2021-01-14 05:07:13.973,"POLYGON ((937416.848 143769.832, 937400.769 14..."
1,2,6251,M033-BLG1531,1081934,1003400001,445450,16.0,39.23,1900.0,1992.0,None,U,current,2021-01-14 05:07:13.973,"POLYGON ((989434.660 201650.170, 989426.582 20..."
2,3,5345,M107-BLG1574,1088078,1018190010,446387,24.0,49.00,1961.0,NaN,None,U,current,2021-01-14 05:07:13.973,"POLYGON ((1004143.512 226698.919, 1004080.866 ..."
3,4,6713,X092-ZN03-BLG0965,2123969,2059000150,449043,22.0,8.97,1970.0,NaN,None,U,current,2021-01-14 05:07:13.973,"POLYGON ((1012801.678 263654.846, 1012779.208 ..."
4,5,4859,Q012B-BLG0580,4560693,4060720001,449179,74.0,10.00,1955.0,NaN,None,U,current,2021-01-14 05:07:13.973,"POLYGON ((1048594.670 221387.270, 1048574.380 ..."


In [19]:
# struct_deltas[struct_deltas['SYSTEM']=='Q015-ZN06-BLG0418']

In [20]:
multipolygons = []

for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
        multipolygons.append(row['SYSTEM'])

if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons)
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
    
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

In [21]:
multipolygons

[]

## Write Delta Table to geojson file

In [22]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\Structures\\delta_snapshots/' + today

In [23]:
if not struct_deltas.empty:
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

## Read geojson file to geojson object 

In [24]:
if not struct_deltas.empty:
    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

## Create arcgis featureSet from geojson object

In [25]:
type(struct_deltas)

geopandas.geodataframe.GeoDataFrame

In [26]:
struct_deltas.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: USA - New York - SPCS - Long Island
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [27]:
# fromDF_deltas = FeatureSet.from_dataframe(pd.DataFrame(struct_deltas))

In [28]:
if not struct_deltas.empty:
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

C:\ProgramData\Anaconda3\envs\structures\lib\site-packages\arcgis\features\feature.py:799: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  warn("crs has been deprecated and will be ignored. Please"+ \


In [29]:
#Check the WKT Projection for the feature set
if len(fromJSON_deltas) > 0:
    print(fromJSON_deltas.spatial_reference)

{'wkid': 4326}


In [30]:
#Set the WKT projection for each feature in the feature set
for f in range(0, len(fromJSON_deltas)):
    fromJSON_deltas.features[f].geometry.update({'spatialReference': {'wkid': 2263}})

In [31]:
#Set the WKT projection for the whole feature set
fromJSON_deltas.spatial_reference = {'wkid': 2263}

In [32]:
#Check the WKT Projection for the feature set
if len(fromJSON_deltas) > 0:
    print(fromJSON_deltas.spatial_reference)

{'wkid': 2263}


# Connect to published datasets via GIS object

In [33]:
gis = GIS(url=portal, verify_cert = False)

# Make Edits

## Structures

### run delta archive stored proc

In [34]:
crsr = con.cursor()
sqal_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

### Connect to structures feature layer directly

In [35]:
strct_lyr_url = structures_url

In [36]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [37]:
structures_features

<FeatureSet> 2707 features

In [38]:
len(struct_deltas)

852

### EDIT

In [39]:
if not struct_deltas.empty:
    num_updates = len(fromJSON_deltas.features)

In [40]:
if not struct_deltas.empty:
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_structures_mssg.txt', subject = 'structures push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

Dear Julie, 

This is a courtesy message to let you know updates to the Structure layer were pushed successfully.

Details:

There were 852 updates pushed to Production.

Thanks,
Data team at IPM



In [41]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

## Geosupport

### connect to geosupport service

In [42]:
geosupport_tbl_url = geosupport_url

In [43]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

### Get latest BINs from structures layer

In [44]:
# structures_valid_BINs = structures_features.sdf[
#     (structures_features.sdf['BIN'] != 0) &
#     (~pd.isnull(structures_features.sdf['BIN']))] 

In [45]:
# bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int) %
#                              100000 != 0]['BIN'].astype(int).to_list()

In [46]:
# bins = [str(bins[i]) for i in range(0, len(bins))]

In [47]:
structures_sdf = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))] 

In [48]:
structs_sdf_final = structures_sdf[structures_sdf['BIN'].astype(int) % 1000000 != 0]

### run master geosupport function

In [49]:
structs_sdf_final.head()

,OBJECTID,SYSTEM,created_user,created_date,last_edited_user,last_edited_date,GlobalID,FEATURESTATUS,FEATURESTATUSCHANGEDATE,BIN,...,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,DOITT_ID,DOITT_Source,ActualSqFeetAboveGrade,ActualSqFeetBelowGrade,SHAPE
0,4519,Q389-BLG0001,DPR,2015-06-11 13:06:36,NYCDPR\scott.gilbert,2020-09-16 17:53:03,{532937F3-5C74-4709-A848-D0A721D73609},Active,NaT,4592110.0,...,7.0,8.730474,1958.0,NaN,None,277465.0,current,NaN,None,"{""rings"": [[[1032852.2682688832, 152292.274533..."
1,4520,Q458-BLG0077,DPR,2015-06-11 13:06:36,NYCDPR\py_services,2021-01-15 04:07:05,{47FD958F-CD1F-4F89-A61B-5CA55E7D7D3A},Inactive,2013-10-04,4453939.0,...,29.0,34.000000,NaN,NaN,None,246576.0,current,NaN,None,"{""rings"": [[[1047002.3764515519, 227118.998239..."
2,4521,Q458-BLG0078,DPR,2015-06-11 13:06:36,NYCDPR\py_services,2021-01-15 04:07:05,{BE6587F7-189E-4A66-9EF5-17D8972976A5},Active,NaT,4539845.0,...,29.0,9.000000,NaN,NaN,None,486282.0,current,NaN,None,"{""rings"": [[[1046728.9501611441, 228000.653300..."
5,4524,Q458-BLG0082,DPR,2015-06-11 13:06:36,NYCDPR\py_services,2021-01-15 04:07:05,{18AA7EB3-EF58-48BE-9A0E-836A19F4B626},Inactive,2013-10-04,4539831.0,...,26.0,42.000000,NaN,NaN,None,827172.0,current,NaN,None,"{""rings"": [[[1046510.868904382, 228830.6959315..."
8,4527,Q458-BLG0085,DPR,2015-06-11 13:06:36,NYCDPR\py_services,2021-01-15 04:07:05,{02EAB4BE-27FC-44CA-B125-2C97ED8E07D0},Inactive,2013-10-04,4539891.0,...,8.0,36.000000,NaN,NaN,None,415163.0,current,NaN,None,"{""rings"": [[[1044900.0824277997, 228794.469954..."


In [50]:
df = master_geosupport_func2(structs_sdf_final, 'BIN')

In [51]:
df.head()

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,...,out_san_dist_section,out_san_recycle,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id,in_boro,Boro_Code
0,4453939,132,132,SERGEANT BEERS LANE,QUEENS,5163967,1.0,NaN,NaN,NaN,...,407 / 07,,,,BAYSIDE,46189701,Address is on the left when facing from GENERA...,0103892,4,4
1,4539831,411,411,SHORE ROAD,QUEENS,5164307,1.0,NaN,NaN,NaN,...,407 / 07,,,,BAYSIDE,46209501,Address is on the right when facing from ORDNA...,0103395,4,4
2,4539848,619,619,LITTLE BAY ROAD,QUEENS,5164224,1.0,NaN,NaN,NaN,...,407 / 07,,,,BAYSIDE,45410701,Address is on the right when facing from WILLE...,0103838,4,4
3,4539879,308,308,RED CROSS LANE,QUEENS,5164008,1.0,NaN,NaN,NaN,...,407 / 07,,,,BAYSIDE,46007701,Address is on the left when facing from BEND t...,0103884,4,4
4,4539878,307,307,SHORE ROAD,QUEENS,5164308,1.0,NaN,NaN,NaN,...,407 / 07,,,,BAYSIDE,46209501,Address is on the right when facing from STORY...,0103398,4,4


### prepare latest geosupport data to be pushed to prod

In [52]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               #'in_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_com_dist': 'Community_Board',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'out_san_sched': 'Sanitation_Subsect',
                               'out_san_dist_section': 'Sanitation_District',
                               'out_san_recycle': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID',
                               'address_id': 'Address_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup', 
                           'Address_ID', 'official_address', 'posted_address'])
       .drop_duplicates())

In [53]:
len(dff)

3880

In [54]:
dff.head(10)

,BIN,Boro_Code,Address_Type,Low_House_Num,High_House_Num,Norm_Street_Name,USPS_City,Zip_Code,Physical_ID,B7SC,...,Police_Boro,Police_Boro_Com,Police_Precinct,Sanitation_Subsect,Sanitation_District,Sanitation_Recycling,Sanitation_Reg_Pickup,Address_ID,official_address,posted_address
0,4453939,4,NaN,132,132,SERGEANT BEERS LANE,BAYSIDE,11359,0103892,NaN,...,Queens North,NaN,109,,407 / 07,,,5163967,0.0,1.0
1,4539831,4,NaN,411,411,SHORE ROAD,BAYSIDE,11359,0103395,NaN,...,Queens North,NaN,109,,407 / 07,,,5164307,0.0,1.0
2,4539848,4,NaN,619,619,LITTLE BAY ROAD,BAYSIDE,11359,0103838,NaN,...,Queens North,NaN,109,,407 / 07,,,5164224,0.0,1.0
3,4539879,4,NaN,308,308,RED CROSS LANE,BAYSIDE,11359,0103884,NaN,...,Queens North,NaN,109,,407 / 07,,,5164008,0.0,1.0
4,4539878,4,NaN,307,307,SHORE ROAD,BAYSIDE,11359,0103398,NaN,...,Queens North,NaN,109,,407 / 07,,,5164308,0.0,1.0
5,4453948,4,NaN,415,415,WALTER REED ROAD,BAYSIDE,11359,0103869,NaN,...,Queens North,NaN,109,,407 / 07,,,5163949,0.0,1.0
6,4297581,4,NaN,16-26,16-26,CROSS BAY BOULEVARD,FAR ROCKAWAY,11693,,NaN,...,Queens South,NaN,100,2C,414 / 142,ETH,MTH,330544,0.0,1.0
7,4592679,4,NaN,80-98,80-98,LINDEN BOULEVARD,HOWARD BEACH,11414,0076911,NaN,...,Queens South,NaN,106,1D,410 / 101,ET,TF,10173282,0.0,1.0
8,4094682,4,NaN,82-54,82-54,88 PLACE,RIDGEWOOD,11385,0097947,NaN,...,Queens North,NaN,104,4E,405 / 054,EW,WS,105020,0.0,1.0
9,4116653,4,NaN,51-04,51-04,KISSENA BOULEVARD,FLUSHING,11355,0005510,NaN,...,Queens North,NaN,109,2D,407 / 072,ET,TF,128838,0.0,1.0


In [55]:
#If the length of the updated dateframe is greater than 0 then truncate the existing table
if len(dff) > 0:
    crsr = con.cursor()
    crsr.execute('''truncate table structuresdb.dbo.tbl_geosupport_address''')
    crsr.commit()
    crsr.close()

In [56]:
dff.to_sql('tbl_geosupport_address',
           engine,
           schema='dbo',
           if_exists='append',
           index=False)

In [57]:
dff.Boro_Code.fillna(0,inplace=True)

In [58]:
dff.Address_ID.fillna(0,inplace=True)

In [59]:
dff = dff.drop(columns = ['official_address', 'posted_address'])

In [60]:
geosupport_FSET = FeatureSet.from_dataframe(dff)

In [61]:
len(geosupport_FSET.features)

3880

### EDIT

In [62]:
if len(dff) > 0: #len(geosupport_FSET.features):
    try:
        ## TRUNCATE AND REPLACE
        tbl_geosupport.delete_features(where="objectid > 0")
        add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)
    except:
        error_mssg = 'Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(len(dff)) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_geosupport_mssg.txt', subject = 'geosupport push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the geosupport table.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport: NO UPDATES to push', e=mssg)

Dear Julie, 

Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.

Thanks,
Data team at IPM

